# Dev Notes

Create & save new Influx DB measures:
- VerticalLine
- HorizontaLine

These will need to include start and stop variables so that we don't have lines into perpetuity

In [1]:
# Required for PydanticAI to work with Jupyter (nested event loops)
import nest_asyncio

nest_asyncio.apply()

In [ ]:
import logging
import asyncio
import pandas as pd
import polars as pl
from zoneinfo import ZoneInfo

from datetime import datetime, timedelta, timezone

import influxdb_client

from tastytrade.connections import InfluxCredentials

from tastytrade.config import RedisConfigManager
from tastytrade.providers.market import MarketDataProvider
from tastytrade.providers.subscriptions import RedisSubscription

from tastytrade.messaging.models.events import CandleEvent

from tastytrade.analytics.visualizations.plots import (
    plot_macd_with_hull,
    HorizontalLine,
    VerticalLine,
)
from tastytrade.analytics.indicators.momentum import macd

In [ ]:
# Show all rows in pandas DataFrames
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

logging.getLogger().handlers.clear()

TEST = True
ENV = "Live"
DURATION = 15

EDT = 5

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s:%(name)s:%(lineno)d:%(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

loop = asyncio.get_event_loop()
loop.set_debug(True)
logging.getLogger("asyncio").setLevel(logging.DEBUG)

# Market Data Subscriptions

In [4]:
config = RedisConfigManager(env_file="/workspace/.env")
config.initialize()

influxdb = influxdb_client.InfluxDBClient(
    url=InfluxCredentials(config=config).url,
    token=InfluxCredentials(config=config).token,
    org=InfluxCredentials(config=config).org,
)

subscription = RedisSubscription(config=RedisConfigManager())
await subscription.connect()

streamer = MarketDataProvider(subscription, influxdb)

2026-02-10 04:03:34 - INFO:tastytrade.config.manager:174:Initialized 52 variables from .env file in Redis
2026-02-10 04:03:34 - INFO:tastytrade.providers.subscriptions:72:Listening to Redis at redis://redis:6379/0


# Date Setup

In [5]:
# Date setup is handled in the next cell

In [6]:
# Use ET timezone
date = datetime.now(ZoneInfo("America/New_York"))

# Roll back to prior day if pre-market
if date < datetime(date.year, date.month, date.day, 9, 30, tzinfo=date.tzinfo):
    date -= timedelta(days=1)

# Roll back to prior Friday if weekend
while date.weekday() > 4:
    date -= timedelta(days=1)

market_open = datetime(date.year, date.month, date.day, 9, 30, tzinfo=date.tzinfo)
morning_end = datetime(date.year, date.month, date.day, 11, 30, tzinfo=date.tzinfo)
lunch_end = datetime(date.year, date.month, date.day, 13, 30, tzinfo=date.tzinfo)
market_close = datetime(date.year, date.month, date.day, 16, 0, tzinfo=date.tzinfo)

start = market_open.astimezone(timezone.utc) - timedelta(minutes=30)
stop = market_close.astimezone(timezone.utc)

In [ ]:
candle_symbol = "SPX{=m}"
# candle_symbol = "BTC/USD:CXTALP{=m}"

# Roll back to prior trading day (skip weekends)
prior_date = market_open.date() - timedelta(days=1)
while prior_date.weekday() > 4:
    prior_date -= timedelta(days=1)

prior_day: CandleEvent = streamer.get_daily_candle(candle_symbol, prior_date)

In [8]:
or5 = await get_opening_range(
    streamer,
    "SPX{=m}",
    5,
    date=start.date(),
)

or15 = await get_opening_range(
    streamer,
    "SPX{=m}",
    15,
    date=start.date(),
)

or30 = await get_opening_range(
    streamer,
    "SPX{=m}",
    30,
    date=start.date(),
)

2026-02-10 04:03:34 - INFO:tastytrade.analytics.visualizations.utils:96:Opening range for SPX{=m} (5m): High=6924.59, Low=6905.87
2026-02-10 04:03:34 - INFO:tastytrade.analytics.visualizations.utils:96:Opening range for SPX{=m} (15m): High=6938.3, Low=6905.87
2026-02-10 04:03:34 - INFO:tastytrade.analytics.visualizations.utils:96:Opening range for SPX{=m} (30m): High=6938.54, Low=6905.87


In [9]:
levels = []

In [15]:
levels = [
    HorizontalLine(
        price=prior_day.close,
        start_time=market_open,
        color="#FF66FE",  # Orange
        line_dash="dot",
        label_font_size=10.5,
        label="prior close",
        opacity=0.45,
        end_time=market_close,
    ),
    HorizontalLine(
        price=prior_day.high,
        start_time=market_open,
        color="#4CAF50",  # Green
        line_dash="dot",
        label_font_size=10.5,
        label="prior high",
        opacity=0.45,
        end_time=market_close,
    ),
    HorizontalLine(
        price=prior_day.low,
        start_time=market_open,
        color="#F44336",  # Red
        line_dash="dot",
        label_font_size=10.5,
        label="prior low",
        opacity=0.45,
        end_time=market_close,
    ),
    HorizontalLine(
        price=or5.high,
        start_time=market_open,
        end_time=market_close,
        color="#4CAF50",  # Green
        line_dash="solid",
        opacity=0.75,
        label="5min high",
    ),
    HorizontalLine(
        price=or5.low,
        start_time=market_open,
        end_time=market_close,
        color="#4CAF50",  # Green
        line_dash="solid",
        opacity=0.75,
        label="5min low",
    ),
    HorizontalLine(
        price=or15.high,
        start_time=market_open + timedelta(minutes=15),
        end_time=market_close,
        color="#4CAF50",  # Green
        line_dash="solid",
        opacity=0.45 if or15.high != or5.high else 0.0,
        label="15min high",
    ),
    HorizontalLine(
        price=or15.low,
        start_time=market_open + timedelta(minutes=15),
        end_time=market_close,
        color="#4CAF50",  # Green
        line_dash="solid",
        opacity=0.45 if or15.low != or5.low else 0.0,
        label="15min low",
    ),
    HorizontalLine(
        price=or30.high,
        start_time=market_open + timedelta(minutes=30),
        end_time=market_close,
        color="#4CAF50",  # Green
        line_dash="dot",
        opacity=0.45 if or30.high != or15.high else 0.0,
        label="30min high",
    ),
    HorizontalLine(
        price=or30.low,
        start_time=market_open + timedelta(minutes=30),
        end_time=market_close,
        color="#4CAF50",  # Green
        line_dash="dot",
        opacity=0.45 if or30.low != or15.low else 0.0,
        label="30min low",
    ),
]

In [11]:
executions = [
    VerticalLine(
        start_time=datetime(date.year, date.month, date.day, 10, 5, tzinfo=date.tzinfo),
        color="#555555",
        line_dash="dot",
        label="Open",
    ),
    VerticalLine(
        start_time=datetime(date.year, date.month, date.day, 11, 40, tzinfo=date.tzinfo),
        color="#555555",
        line_dash="dot",
        label="Close",
    ),
    VerticalLine(
        start_time=datetime(date.year, date.month, date.day, 12, 25, tzinfo=date.tzinfo),
        color="#555555",
        line_dash="dot",
        label="Open",
    ),
    VerticalLine(
        start_time=datetime(date.year, date.month, date.day, 13, 40, tzinfo=date.tzinfo),
        color="#555555",
        line_dash="dot",
        label="Close",
    ),
]

In [12]:
candles: pl.DataFrame = streamer.download(
    symbol=candle_symbol,
    start=start,
    stop=stop,
    debug_mode=True,
)

# Filter out any rows where an OHLC price is 0 (invalid for active securities)
_initial_len = candles.height
candles = candles.filter(
    (pl.col("open") != 0)
    & (pl.col("high") != 0)
    & (pl.col("low") != 0)
    & (pl.col("close") != 0)
)
_removed = _initial_len - candles.height
print(
    f"Removed {_removed} zero-price rows out of {_initial_len} total ({(_removed / _initial_len * 100) if _initial_len else 0:.2f}%)."
)

df_macd = macd(
    candles, prior_close=prior_day.close, fast_length=12, slow_length=26, macd_length=9
)

plot_macd_with_hull(
    df_macd,
    pad_value=prior_day.close,
    start_time=start,
    end_time=stop + timedelta(minutes=15),
    horizontal_lines=levels,
    vertical_lines=executions,
)

Removed 0 zero-price rows out of 420 total (0.00%).


In [13]:
candles_5m: pl.DataFrame = streamer.download(
    symbol=candle_symbol.replace("m", "5m"),
    start=start,
    stop=stop,
    debug_mode=True,
)

_initial_len_5m = candles_5m.height
candles_5m = candles_5m.filter(
    (pl.col("open") != 0)
    & (pl.col("high") != 0)
    & (pl.col("low") != 0)
    & (pl.col("close") != 0)
)
_removed_5m = _initial_len_5m - candles_5m.height
print(
    f"Removed {_removed_5m} zero-price 5m rows out of {_initial_len_5m} total ({(_removed_5m / _initial_len_5m * 100) if _initial_len_5m else 0:.2f}%)."
)

df_macd_5m = macd(
    candles_5m,
    prior_close=prior_day.close,
    fast_length=12,
    slow_length=26,
    macd_length=9,
)

plot_macd_with_hull(
    df_macd_5m,
    pad_value=prior_day.close,
    start_time=start,
    end_time=stop + timedelta(minutes=15),
    horizontal_lines=levels,
    vertical_lines=executions,
)

Removed 0 zero-price 5m rows out of 84 total (0.00%).


## Persist Annotations to InfluxDB

Write annotations (HorizontalLine / VerticalLine) through the same
`TelegrafHTTPEventProcessor` that handles CandleEvent, TradeEvent, etc.

Annotations are `BaseEvent` subclasses, so `process_event()` serializes
them natively — measurement name from the class, `eventSymbol` as tag,
all other fields as InfluxDB fields.

In [17]:
from tastytrade.messaging.processors.influxdb import TelegrafHTTPEventProcessor

# Extract the base ticker for eventSymbol (e.g. "SPX" from "SPX{=m}")
base_ticker = candle_symbol.split("{=")[0]

# Tag all annotations with the symbol so they're queryable in InfluxDB
all_annotations = levels + executions
tagged = []
for ann in all_annotations:
    if not ann.eventSymbol:
        tagged.append(ann.model_copy(update={"eventSymbol": base_ticker}))
    else:
        tagged.append(ann)

# Write through the same processor pipeline as CandleEvent / TradeEvent
processor = TelegrafHTTPEventProcessor()
for annotation in tagged:
    processor.process_event(annotation)
processor.close()

print(f"Persisted {len(tagged)} annotations to InfluxDB for {base_ticker}")
print(f"  HorizontalLine: {sum(1 for a in tagged if isinstance(a, HorizontalLine))}")
print(f"  VerticalLine:   {sum(1 for a in tagged if isinstance(a, VerticalLine))}")

2026-02-10 04:17:20 - INFO:tastytrade.messaging.processors.influxdb:44:Flushing InfluxDB write API...
2026-02-10 04:17:20 - INFO:tastytrade.messaging.processors.influxdb:47:InfluxDB client closed


Persisted 13 annotations to InfluxDB for SPX
  HorizontalLine: 9
  VerticalLine:   4


### Redis Candle Feed Monitor (1m and 5m)

Manual workflow to inspect live CandleEvent data at 1m ("m") and 5m intervals:

1. Run the Subscription Setup cell (next) once to ensure Redis subscriptions are active.
2. Run the Manual Poll cell any time you want to pull newly arrived events off the queues.
3. (Optional) Re-run the DataFrame Summary cell to view/aggregate everything collected so far.

No background loop or timeout is used; you stay in control of when data is fetched.

In [ ]:
# import datetime, polars as pl
# from tastytrade.providers.subscriptions import RedisSubscription
# from tastytrade.messaging.models.events import CandleEvent

# SYMBOL = candle_symbol if "candle_symbol" in globals() else "SPX{=1m}"
# base_ticker = SYMBOL.split("{=")[0]
# intervals = ["m", "5m"]
# patterns = [f"market:CandleEvent:{base_ticker}{{={iv}}}" for iv in intervals]
# print(f"Ensuring subscriptions active for: {patterns}")

# # Reuse existing subscription if possible
# redis_sub: RedisSubscription
# if "subscription" in globals() and isinstance(subscription, RedisSubscription):
#     redis_sub = subscription
#     if not getattr(redis_sub, "pubsub", None):
#         await redis_sub.connect()
# else:
#     redis_sub = RedisSubscription(config)
#     await redis_sub.connect()

# # Subscribe (idempotent)
# for p in patterns:
#     await redis_sub.subscribe(p)

# # Prepare/restore collection list
# if "collected" not in globals():
#     collected: list[CandleEvent] = []  # type: ignore

# print("Subscription setup complete. Use the next cell to manually poll for new events.")

In [ ]:
# # Manual Poll: run this cell whenever you want to retrieve any newly queued CandleEvents
# from tastytrade.messaging.models.events import CandleEvent

# base_ticker = SYMBOL.split("{=")[0]
# new_events = 0
# for iv in intervals:
#     queue_key = f"CandleEvent:{base_ticker}{{={iv}}}"
#     queue = redis_sub.queue.get(queue_key)
#     if queue is None:
#         continue
#     while True:
#         try:
#             evt: CandleEvent = queue.get_nowait()
#             collected.append(evt)
#             new_events += 1
#             print(
#                 f"{evt.eventSymbol} O:{evt.open} H:{evt.high} L:{evt.low} C:{evt.close} @ {evt.tradeDateUTC} {evt.tradeTimeUTC}"
#             )
#         except Exception:
#             break
# print(f"Added {new_events} new events. Total collected: {len(collected)}")

In [ ]:
# # Build a Polars DataFrame from collected CandleEvent objects
# if "collected" in globals() and collected:
#     df = pl.DataFrame(
#         [
#             {
#                 "symbol": e.eventSymbol,
#                 "interval": e.eventSymbol.split("{=")[1].rstrip("}"),
#                 "time": f"{e.tradeDateUTC} {e.tradeTimeUTC}",
#                 "open": e.open,
#                 "high": e.high,
#                 "low": e.low,
#                 "close": e.close,
#                 "volume": getattr(e, "volume", None),
#                 "sequence": getattr(e, "sequence", None),
#             }
#             for e in collected
#         ]
#     )
#     print(df.tail())

#     if df.select(pl.col("interval").n_unique()).item() > 1:
#         agg = (
#             df.group_by("interval")
#             .agg(
#                 [
#                     pl.col("open").last().alias("last_open"),
#                     pl.col("high").max().alias("session_high"),
#                     pl.col("low").min().alias("session_low"),
#                     pl.col("close").last().alias("last_close"),
#                     pl.col("time").last().alias("last_time"),
#                 ]
#             )
#             .sort("interval")
#         )
#         print("\nSummary by interval:")
#         print(agg)
# else:
#     print("No events collected yet. Run the manual poll cell above first.")